In [ ]:
%load_ext autoreload
%autoreload

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

-

Settings

train = 0 </br>
valid = 3 </br>
test = 1 </br>
test' = 2 </br>

In [ ]:
# Тип выборки
MODE = 0
# Номер переменной
VAR = 8
# Число кластеров с наименьшей MAE на рассмотрение
TOP = 8
# Длина участка визуализации
LEN = 10

-

In [ ]:
data = pd.read_csv("./output_table_03-22.csv")
data = data[data["mode"].astype("int") == MODE]

In [ ]:
data.loc[:, ["real 0", "real 23", "real 8", "predicted 0", "predicted 8", "predicted 23", "cluster_num", "mode"]]

Оценим переменные

In [ ]:
real = [x for x in data.columns if "real" in x]
predicted = [x for x in data.columns if "predicted" in x]
err = abs(data[real].values - data[predicted].values)
err = err[data["mode"] == MODE]
pd.DataFrame([dict(zip(real, err.mean(axis=0)))]).T.sort_values(0)[:10]

In [ ]:
data["mode"].value_counts()

In [ ]:
AE = abs(data[f"real {VAR}"] - data[f"predicted {VAR}"])
AE = pd.DataFrame(dict(AE=AE, cluster_num=data["cluster_num"]))

In [ ]:
AE

In [ ]:
top = [AE[AE["cluster_num"] == c]["AE"].mean() for c in np.arange(AE["cluster_num"].max())]
top = np.argsort(top)

In [ ]:
top

-

# Визуализация

In [ ]:
for i in range(min(TOP, len(top))):
    #print(int(f"{i+1}1{TOP}"))
    plt.subplot(int(f"{TOP}1{i+1}"))
    index = AE["cluster_num"] == top[i]
    plt.title(f"Cluster: {top[i]}, MAE: {AE[index]['AE'].mean():.5f}, Size: {sum(AE['cluster_num'] == top[i])}")
    cluster = data[[f"real {VAR}", f"predicted {VAR}"]][index]
    print(cluster.shape)
    # find continous index
    print(cluster.index.shape)
    cont = np.split(cluster, np.where(np.diff(cluster.index.to_numpy()) > 1)[0] + 1)
    print([len(x) for x in cont])
    ind = np.argmax([len(x) for x in cont])
    chunk = [x for x in cont if len(x) > LEN][-1]
#     chunk = cont[ind]
    start = LEN * 0 // 5
    plt.plot(chunk[start:start+5*LEN])
    plt.legend(chunk.columns)
    
plt.gcf().set_size_inches(15, 5 * TOP)
# plt.savefig("top_clusters_visualize.png")
plt.show()